<a href="https://colab.research.google.com/github/stixmal/praktikum_project_ds/blob/main/bank_borrowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



  # Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Источник: `/datasets/data.csv`.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавление  

### 1. Ознакомление с общей информацией


### 2. Предобработка данных
+ Работа с пропусками  


+ Замена типа данных  


+ Обработка дубликатов  


+ Лемматизация


+ Категоризация данных
    
    
### 3. Ответы на вопросы  
+ Влияние наличия детей на возврат кредита в срок


+ Влияние семейного положения на возврат кредита в срок


+ Корелляция между уровнем дохода и возвратом кредита в срок


+ Влияние целей кредита на его возврат в срок


### 4. Общий вывод  
***
 

## 1. Ознакомление с общей информацией

Изучим данные, предоставленные банком для проекта.

In [ ]:
# импорт библиотеки pandas
import pandas as pd     

# импорт модуля display
from IPython.display import display    

Прочитаем файл, предоставленный банком, и сохраним его в переменной `df`.

In [ ]:
# чтение файла с данными
df = pd.read_csv('/datasets/data.csv')  

Посмотрим визуально на таблицу.

In [ ]:
# вызов таблицы на экран
display(df)     

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Общая информация о данных таблицы *'df'*.

In [ ]:
# получение общей информации о данных в таблице df
df.info()           

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную таблицу.

Всего в таблице 12 столбцов с целочисленными, вещественными и строковыми данными.

Разберём какю информацию содержат столбцы.

* **children** — количество детей в семье
* **days_employed** — общий трудовой стаж в днях
* **dob_years** — возраст клиента в годах
* **education** — уровень образования клиента
* **education_id** — идентификатор уровня образования
* **family_status** — семейное положение
* **family_status_id** — идентификатор семейного положения
* **gender** — пол клиента
* **income_type** — тип занятости
* **debt** — имел ли задолженность по возврату кредитов
* **total_income** — ежемесячный доход
* **purpose** — цель получения кредита

Количество значений  в столбцах различается. Это указывает на наличие пропущенных значений.

###  Выводы

Первой проблемой стало обнаружение пропусков в нашей таблице.Устраним её в дальнейшем. Для ответов на поставленные вопросы наиболее ценны столбцы *'children'*, *'family_status'*, *'total_income'*, *'purpose'*, *'debt'*.

***

## 2. Предобработка данных

* ### Работа с пропусками

В столбцах *'days_employed'* и *'total_income'* с вещественными данными обнаружили пропуски. 

In [ ]:
# Подсчёт количества пропусков в таблице 'df'
df.isnull().sum()  

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Визуально в столбце с общим трудовым стажем в днях мы видим отрицательные значения, что не соответствует действительности. Приведём их к абсолютному виду.

In [ ]:
# Приведение отрицательных значений в абсолютную форму
df['days_employed'] = abs(df['days_employed'])   

Так как в обоих столбцах находятся количественные переменные, можно применить метод заполнения пропусков характерными значениями. Однако в столбце *'days_employed'* имеются значения-артефакты, которые кардинально выделяются и не отвечают здравому смыслу. Причины возникновения таких данных требует дополнительной проверки.



In [ ]:
# Сортировка столбца 'days_employed' по убыванию
df['days_employed'].sort_values(ascending = False)   

6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
             ...      
21489              NaN
21495              NaN
21497              NaN
21502              NaN
21510              NaN
Name: days_employed, Length: 21525, dtype: float64

In [ ]:
# Нахождение среднего в столбце 'days_employed'
df['days_employed'].mean()  

66914.72890682236

In [ ]:
# Нахождение медианы в столбце 'days_employed'
days_employed_median = df['days_employed'].median()  
days_employed_median

2194.220566878695

В качестве метода замены использовали медианное значение, так как оно более точно характеризует выборку при наличии данных существенно отклоняющихся от большинства.

Выполнили замену пропусков в столбце *'days_employed'*.

In [ ]:
# Замена пропусков медианным значением
df['days_employed'] = df['days_employed'].fillna(value = days_employed_median)     

Второй столбец с пропущенными значениями *'total_income'* также заменили медианным значением.

In [ ]:
# Нахождение медианы в столбце 'total_income'
total_income_median = df['total_income'].median()           

# Замена пропусков медианным значением
df['total_income'] = df['total_income'].fillna(value = total_income_median)  

Получаем общую информацию о таблице и проверяем результат замены.

In [ ]:
# Общая информация о таблице `df`
df.info()       

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


###  Выводы

Выполнили поиск и замену пропусков в таблице *'df'*. В дальнейшем это позволит точнее провести анализ.

На этапе работы с пропусками обнаружили несколько проблем с вводными данными, такими как:

+ отрицательные значения трудового стажа в днях
+ данные-артефакты, которые не отражают действительность (количество дней трудового стажа гораздо больше величины человеческой жизни)

Данные в этих двух столбцах представлены  в вещественной форме, что может косвенно указывать на ошибки их внесения компьютерными системами.



* ### Замена типа данных

Заменили вещественный тип данных на целочисленный методом `.astype`.

In [ ]:
# Замена типа данных на целочисленный в столбце 'days_employed'
df['days_employed'] = df['days_employed'].astype('int')     

# Замена типа данных на целочисленный в столбце 'total_income'
df['total_income'] = df['total_income'].astype('int')       

Посмотрели результат замены, вызвав метод `.info`

In [ ]:
# Общая информация о таблице `df`
df.info()        

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


###  Выводы

Заменили вещественный тип данных на целочисленный методом `.astype`. Данный метод выбрали в виду отсутствия строковых данных в заменяемых столбцах.

* ### Обработка дубликатов

Посмотрели на наличие уникальных значений интересующих нас столбцов, которые необходимы для ответов на поставленные вопросы.

In [ ]:
# уникальные значения столбца 'children'
df['children'].value_counts()    

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Так как нам нужен сам факт наличия детей, а не их количество, заменили некорректные данные условно: отрицательные значения на **1**, нереальное количество детей на **2**. Данные несоответсвия возможны в одном случае случайным нажатием нуля в виду близкого расположения к двойке, а в другом наличия предустановленного дефиса.

In [ ]:
# замена строк в столбце 'children'
df.loc[df['children'] == -1, 'children'] = 1    

df.loc[df['children'] == 20, 'children'] = 2

Проверяем наш новый столбец.

In [ ]:
# проверка столбца
df['children'].value_counts()   

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

***

Также смотрим на столбец *'dob_years'*

In [ ]:
df['dob_years'].unique() 

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Видим, что есть выпадающее значение: запись с нулевым значением. Заменили округлённым средним методом `.mean` и функцией `round`, так как разброс величин возрастов конечен и не выходит за рамки жизни человека.

In [ ]:
# нахождение строки логической индексацией и замена неправильного значения округлённым средним

df.loc[df['dob_years'] == 0, 'dob_years'] = round(df['dob_years'].mean())

# проверка значений столбца 'dob_years'

df['dob_years'].unique() 

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

***

Уникальные строки в столбце *'education'*.

In [ ]:
# подсчет уникальных значений столбца 'education'
df['education'].value_counts() 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Привели к нижнему регистру методом `str.lower`.

In [ ]:
#  перевод строковых данных в столбце к нижнему регистру
df['education'] = df['education'].str.lower()   

# проверка исправленного столбца

df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Посмотрели на сгруппированные данные с 'education_id'.

In [ ]:
# отсортированная таблица по убыванию с посчитанным количеством строк, приходящихся на группированные столбцы

df.groupby(['education_id', 'education'])['education'].count().sort_values(ascending = False) 

education_id  education          
1             среднее                15233
0             высшее                  5260
2             неоконченное высшее      744
3             начальное                282
4             ученая степень             6
Name: education, dtype: int64

Каждому типу образования соответствует определённый балл. Возможно данное оценивание помогает в анализе кредитоспособности заёмщика.

***

Посмотрели количество уникальных записей в столбце семейного положения.

In [ ]:
# подсчет уникальных значений столбца 'family_status'
df['family_status'].value_counts() 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

На первый взгляд схожие категории данных не объединены, так как человек либо в браке, либо нет. Посмотрели на сгруппированные данные с *'family_id'*.

In [ ]:
# отсортированная таблица по убыванию с посчитанным количеством строк, приходящихся на группированные столбцы

df.groupby(['family_status_id', 'family_status'])['family_status_id'].count().sort_values(ascending = False) 

family_status_id  family_status        
0                 женат / замужем          12380
1                 гражданский брак          4177
4                 Не женат / не замужем     2813
3                 в разводе                 1195
2                 вдовец / вдова             960
Name: family_status_id, dtype: int64

Мы видим, что данные в столбцах идентичны и не имеют расхождений. Возможно это преднамеренное разделение и выставление баллов в зависимости от степени наличия родственных связей.


***



В столбце *'gender'* удалили методом `.drop` строку с непонятным смыслом.


In [ ]:
# подсчет уникальных значений столбца 'gender'
df['gender'].value_counts() 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [ ]:
# удаление строки методом drop и сброс индексов у таблицы
df = df.drop(df[df.gender == 'XNA'].index).reset_index(drop = True) 

# проверка удаления строки
df['gender'].value_counts() 

F    14236
M     7288
Name: gender, dtype: int64

***

Удалили строки в столбце *'income_type'*, имеющие небольшое количество записей и которые сложно дифференцировать по основным группам.

In [ ]:
# подсчет уникальных значений столбца 'income_type'
df['income_type'].value_counts()  

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [ ]:
# удаление строк методом .drop с логическим условием и сброс индексов у таблицы df 

df = df.drop(df[(df.income_type == 'безработный') | (df.income_type == 'предприниматель') \
                | (df.income_type == 'студент') | (df.income_type == 'в декрете')].index).reset_index(drop = True)


# проверка столбца 'income_type'
df['income_type'].value_counts()

сотрудник      11119
компаньон       5084
пенсионер       3856
госслужащий     1459
Name: income_type, dtype: int64

***

Посмотрели на столбец *'debt'* и поняли, что он не имеет неточностей.

In [ ]:
# подсчет уникальных значений столбца 'debt'
df['debt'].value_counts() 

0    19779
1     1739
Name: debt, dtype: int64

###  Выводы

При поиске дубликатов мы обнаружили разные несоответвия и нехарактерные данные в столбцах нашей таблицы. При приведении в норму использовали различные методы: 

+ замена характерными значениями: близкие по смыслу, усредненное среднее по выборке

+ приведение к нижнему регистру строковых данных

+ удаление строк

Таблица практически очищена от неточностей и готова к дальнейшему анализу.

* ### Лемматизация

Произвели лемматизацию столбца с целями получения кредита.

In [ ]:
# импорт библиотеки pymystem3 с функцией лемматизации на русском языке
from pymystem3 import Mystem        
m = Mystem()

# импорт 'счётчика' из модуля collections
from collections import Counter     


Написали функцию для лемматизации столбца *'purpose'* и создали столбец методом `.apply`.

In [ ]:
# функция для лемматизации столбца с выводом строк лемм
def to_lemmas(text):                                         
    lemmas = m.lemmatize(text)
    lemmas_str = ' '.join(m.lemmatize(text))
    return lemmas_str

# создание столбца с леммами в таблице df
df['lemmas_purpose'] = df['purpose'].apply(to_lemmas)         
df['lemmas_purpose']

0                         покупка   жилье \n
1               приобретение   автомобиль \n
2                         покупка   жилье \n
3            дополнительный   образование \n
4                       сыграть   свадьба \n
                        ...                 
21513                операция   с   жилье \n
21514             сделка   с   автомобиль \n
21515                        недвижимость \n
21516    на   покупка   свой   автомобиль \n
21517           на   покупка   автомобиль \n
Name: lemmas_purpose, Length: 21518, dtype: object

In [ ]:
# визуальная оценка смысловых лемм в столбце 'purpose'
df['lemmas_purpose'].value_counts()             

автомобиль \n                                    972
свадьба \n                                       797
на   проведение   свадьба \n                     776
сыграть   свадьба \n                             774
операция   с   недвижимость \n                   676
покупка   коммерческий   недвижимость \n         664
операция   с   жилье \n                          653
покупка   жилье   для   сдача \n                 652
операция   с   коммерческий   недвижимость \n    651
покупка   жилье \n                               647
жилье \n                                         647
покупка   жилье   для   семья \n                 641
строительство   собственный   недвижимость \n    634
недвижимость \n                                  634
операция   со   свой   недвижимость \n           630
строительство   жилой   недвижимость \n          626
покупка   недвижимость \n                        623
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

###  Выводы

В процессе лемматизации столбца с целями получения кредита мы обнаружили, что все цели определяются очень ограниченным кругом интересов. При визуальной оценке эти интересы можно условно распределить по категориям:

* недвижимость
* автомобиль
* образование
* свадьба

В дальнейшем категоризируем эти данные для более удобного анализа.

* ### Категоризация данных

###  *'debt'*

Функция для распределения факта наличия задолженности по кредиту.

In [ ]:
# функция распределения факта наличия задолженности по категориям
def to_cat_debt(row):                                     
    if row == 0:
        return 'не было'
    else:
        return 'была задолженность'

    # создание столбца с категориями в таблице df
df['category_debt'] = df['debt'].apply(to_cat_debt)     
df['category_debt']    

0                   не было
1                   не было
2                   не было
3                   не было
4                   не было
                ...        
21513               не было
21514               не было
21515    была задолженность
21516    была задолженность
21517               не было
Name: category_debt, Length: 21518, dtype: object

***

###  *'purpose'*

Написали функцию для столбца с леммами, распределяющую их по категориям, и создали столбец методом `.apply`.

In [ ]:
# функция распределения целей получения кредита по категориям
def to_cat_purpose(row):                                         

    if ('недвижимость' in row.split(' ')) | ('жилье' in row.split(' ')):
        return 'недвижимость'
    if 'автомобиль' in row.split(' '):
        return 'автомобиль'
    if 'образование' in row.split(' '):
        return 'образование'
    if 'свадьба' in row.split(' '):
        return 'свадьба'

# создание столбца с категориями в таблице df
df['category_purpose'] = df['lemmas_purpose'].apply(to_cat_purpose)     
df['category_purpose']

0        недвижимость
1          автомобиль
2        недвижимость
3         образование
4             свадьба
             ...     
21513    недвижимость
21514      автомобиль
21515    недвижимость
21516      автомобиль
21517      автомобиль
Name: category_purpose, Length: 21518, dtype: object

Посмотрели распределение значений нового столбца.

In [ ]:
# уникальные значения столбца 'category_purpose'
df['category_purpose'].value_counts()        

недвижимость    10835
автомобиль       4314
образование      4022
свадьба          2347
Name: category_purpose, dtype: int64

Создали сводную таблицу для просмотра влияния целей получения кредита на его возврат в срок.

In [ ]:
# создание сводной таблицы
df_pivot_purpose = df.pivot_table(index=['category_purpose'], columns='category_debt', values='total_income', aggfunc='count')

# создание столбца с отношениями категорий к факту наличия или отсутствия задолженности
df_pivot_purpose['ratio_%'] = 100 * df_pivot_purpose['была задолженность'] / df_pivot_purpose['не было']

df_pivot_purpose.sort_values('ratio_%', ascending = False)

category_debt,была задолженность,не было,ratio_%
category_purpose,,,
автомобиль,402,3912,10.276074
образование,370,3652,10.131435
свадьба,186,2161,8.607126
недвижимость,781,10054,7.768053


***

###  *'children'*

Функция для распределения количества детей по категориям.

In [ ]:
# функция распределения наличия детей по категориям
def to_cat_child(row):                                           
    if row == 0:
        return 'отсутствуют'
    if row != 0:
        return 'есть дети'

# создание столбца с категориями в таблице df    
df['category_children'] = df['children'].apply(to_cat_child)     
df['category_children']

0          есть дети
1          есть дети
2        отсутствуют
3          есть дети
4        отсутствуют
            ...     
21513      есть дети
21514    отсутствуют
21515      есть дети
21516      есть дети
21517      есть дети
Name: category_children, Length: 21518, dtype: object

Создали сводную таблицу для просмотра влияния наличия детей на возврат кредита в срок.

In [ ]:
# создание сводной таблицы
df_pivot_children = df.pivot_table(index=['category_children'], columns='category_debt', values='total_income', aggfunc='count')

# создание столбца с отношениями категорий к факту наличия или отсутствия задолженности
df_pivot_children['ratio_%'] = 100 * df_pivot_children['была задолженность'] / df_pivot_children['не было']
df_pivot_children.sort_values('ratio_%', ascending = False)

category_debt,была задолженность,не было,ratio_%
category_children,,,
есть дети,676,6698,10.092565
отсутствуют,1063,13081,8.126290


***

###  *'family_status'*

Функция для распределения семейного положения по категориям.

In [ ]:
# функция распределения семейного положения по категориям
def to_cat_family(row):                                               
    if row == 0:
        return 'в браке'
    else:
        return 'свободен'
    
# создание столбца с категориями в таблице df    
df['category_family'] = df['family_status_id'].apply(to_cat_family)     
df['category_family']

0         в браке
1         в браке
2         в браке
3         в браке
4        свободен
           ...   
21513    свободен
21514     в браке
21515    свободен
21516     в браке
21517     в браке
Name: category_family, Length: 21518, dtype: object

Создали сводную таблицу для просмотра влияния семейного положения на возврат кредита в срок.

In [ ]:
# создание сводной таблицы
df_pivot_family = df.pivot_table(index=['category_family'], columns='category_debt', values='total_income', aggfunc='count')

# создание столбца с отношениями категорий к факту наличия или отсутствия задолженности
df_pivot_family['ratio_%'] = 100 * df_pivot_family['была задолженность'] / df_pivot_family['не было']
df_pivot_family.sort_values('ratio_%', ascending = False)

category_debt,была задолженность,не было,ratio_%
category_family,,,
свободен,810,8331,9.722722
в браке,929,11448,8.114955


***

###  *'total_income'*

Функция для распределения дохода по категориям.

In [ ]:
# функция распределения дохода по категориям
def to_cat_income(row):                                                     
    if row <= 0.75 * df['total_income'].median():
        return 'низкий доход'
    if row > 2 * df['total_income'].median():
        return 'высокий доход'
    else:
        return 'средний доход'
    
# создание столбца с категориями в таблице df    
df['category_total_income'] = df['total_income'].apply(to_cat_income)     
df['category_total_income']    

0        средний доход
1        средний доход
2        средний доход
3        средний доход
4        средний доход
             ...      
21513    средний доход
21514    средний доход
21515     низкий доход
21516    средний доход
21517     низкий доход
Name: category_total_income, Length: 21518, dtype: object

Создали сводную таблицу для просмотра влияния уровня дохода на возврат кредита в срок.

In [ ]:
# создание сводной таблицы
df_pivot_total_income = df.pivot_table(index=['category_total_income'], columns='category_debt', values='total_income', aggfunc='count')

# создание столбца с отношениями категорий к факту наличия или отсутствия задолженности
df_pivot_total_income['ratio_%'] = 100 * df_pivot_total_income['была задолженность'] / df_pivot_total_income['не было']
df_pivot_total_income.sort_values('ratio_%', ascending = False)

category_debt,была задолженность,не было,ratio_%
category_total_income,,,
средний доход,1183,13183,8.973678
низкий доход,436,5062,8.613196
высокий доход,120,1534,7.822686


###  Выводы

При категоризации столбца с целями получения кредита мы обнаружили, что условно интерес в заемных средствах представляет только четыре цели. Все они относятся к долгосрочным и дорогим покупкам и потому требующие кредитных средств банка. 

Категоризация столбца с детьми получила 2 категории: наличия или отсутствия, так как конкретное количество не важно в ответе на поставленный вопрос. 

Также столбец с семейным положением: либо человек в браке, либо свободен, потому что не юридически обязывающие отношения между людьми сомнительно влияют на возврат кредита. 

Категоризацию столбца доходов провели согласно рекомендации ОЭСР.

***

## 3. Ответы на вопросы

* ### Влияние наличия детей на возврат кредита в срок

Процент людей с детьми, невозвращающих кредит, составил 10,1 %. Процент людей без детей, невозвращающих кредит, составил 8,1 %. Различие имеется, но не настолько, чтобы утверждать о явной зависимости наличия детей и возвратом кредита в срок. При любом раскладе процент невозврата будет около 10 %. Поэтому зависимости нет.

* ### Влияние семейного положения на возврат кредита в срок

Семейное положение также не слишком коррелирует с фактом возврата кредита в срок, так как и в этом случае, свободен ли человек или в браке, явно не указывает на его будущую недобросовестность. Различие процента невозврата в категориях семейного положения сотавляет ~ 1,5 %. Поэтому зависимости между семейным положением и возвратом кредита в срок явно не просматривается.

* ### Корелляция между уровнем дохода и возвратом кредита в срок

Процент невозврата кредита у людей с высокими доходами оказывается самым низким. Процент должников со средними и низкими доходами незначительно (на ~ 1 %) превышает процент должников с высокими доходами, что также говорит о отсутствии явной связи между уровнем дохода и возвратом заемных средств в срок.

* ### Влияние целей кредита на его возврат в срок

Процент невозврата показывает, что наибольшую недобросовестность показывают заемщики, целью получения кредита которых являлся автомобиль (10,3 %). Категория недвижимости наоборот показывает меньшую долю невозвратов (7,8 %). Жильё, как часть дорогостоящей недвижимости, является оплотом стабильности жизни человека. Понимая это, заёмщик меньше всего хочет его потерять. Иными словами без автомобиля "прожить можно", а в случае лишения жилья гораздо сложнее.

***

## 4. Общий вывод

В результате исследования надёжности заёмщиков мы выяснили, что наличие детей, семейное положение и уровень доходов практически не влияют на возврат кредита в срок. Отличия разных категорий в доле невозврата составляет приблизительно 1,5 - 2 %, что указывает на слабую корреляцию между разным количеством детей, разным семейным положением, разным доходом и недобросовестностью клиентов. 